In [ ]:
from grad_val import GradTensor
import random
import numpy as np
import torch
import graphviz

In [ ]:
model_size = 3
x_vals = np.array([random.random() for _ in range(model_size)])
param_vals = np.array([random.random() for _ in range(model_size)])
x = np.array([GradTensor(r) for r in x_vals])
params = np.array([GradTensor(r) for r in param_vals])

out = x * params

final = np.sum(out)

final.backward()
final.visualize()

In [16]:
from simple_net import *
from optimizer import GDOptimizer
from loss import MSELoss

BATCH_SIZE = 32
MAX_ITER = 100

y_true = np.array([GradVal(random()) for _ in range(BATCH_SIZE)])
x = np.array([[GradVal(random()) for _ in range(8)] for _ in range(BATCH_SIZE)])

pipeline = [SimpleLinearLayer(8, 16), SimpleReluLayer(), SimpleLinearLayer(16, 8), SimpleReluLayer(), SimpleLinearLayer(8, 1)]
model = Model(pipeline=pipeline)
optimizer = GDOptimizer(model.parameters(), lr=1e-4)
mse = MSELoss()

for i in range(MAX_ITER+1):
    y_pred = model.forward(x)
    loss = mse.loss(y_pred=y_pred, y_true=y_true)
    loss.backward()
    optimizer.optimize()
    loss.zero_grad()
    if i % 25 == 0:
        print(f"MSE Loss: {loss.val:.3f} after {i} iters")


MSE Loss: 2.335 after 0 iters
MSE Loss: 2.034 after 25 iters
MSE Loss: 1.826 after 50 iters


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f02f0304920>>
Traceback (most recent call last):
  File "/home/kevin/anaconda3/envs/dev/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


MSE Loss: 1.678 after 75 iters
MSE Loss: 1.566 after 100 iters


In [1]:
from vector_net import *
from optimizer import GDOptimizer
from loss import MSELoss
from simple_net import Model

BATCH_SIZE = 32
MAX_ITER = 100

y_true = GradVector(np.random.rand(BATCH_SIZE, 1))
x = GradVector(np.random.rand(BATCH_SIZE, 8))

pipeline = [VectorLinearLayer(8, 16), VectorReluLayer(), VectorLinearLayer(16, 8), VectorReluLayer(), VectorLinearLayer(8, 1)]
model = Model(pipeline=pipeline)
optimizer = GDOptimizer(model.parameters(), lr=1e-5)
mse = MSELoss()

for i in range(MAX_ITER):
    y_pred = model.forward(x)
    loss = mse.loss(y_pred=y_pred, y_true=y_true)
    loss.backward()
    optimizer.optimize()
    loss.zero_grad()
    if i % 25 == 0:
        print(f"MSE Loss: {loss.val[0]} after {i} iters")


MSE Loss: [22.01774708] after 0 iters
MSE Loss: [16.53186919] after 25 iters
MSE Loss: [6.55472266] after 50 iters
MSE Loss: [0.81047027] after 75 iters


In [17]:
from vector_net import *
from optimizer import GDOptimizer
from loss import MSELoss
from simple_net import Model
import torch
import numpy as np
from grad_vector import GradVector

BATCH_SIZE = 256
MAX_ITER = 100

mse = MSELoss()

actual = np.random.rand(BATCH_SIZE, 1)
x = np.random.rand(BATCH_SIZE, 8)
layer1 = VectorLinearLayer(8, 16)
layer2 = VectorLinearLayer(16, 1)

t_actual = torch.tensor(actual, requires_grad=True)
data = torch.tensor(x, requires_grad=True)

weights1 = torch.tensor(layer1.weights.val, requires_grad=True)
weights2 = torch.tensor(layer2.weights.val, requires_grad=True)
bias1 = torch.tensor(layer1.bias.val, requires_grad=True)
bias2 = torch.tensor(layer2.bias.val, requires_grad=True)

res = data @ weights1 + bias1
res = res @ weights2 + bias2
torch_mse = mse.loss(res, t_actual)
torch_mse.backward()

res_gv = layer1(GradVector(x))
res_gv = layer2(res_gv)
gv_mse = mse.loss(res_gv, GradVector(actual))
gv_mse.backward()

In [24]:
weights1.grad.allclose(torch.tensor(layer1.weights.gradient) * 2)

True

In [25]:
bias1.grad.allclose(torch.tensor(layer1.bias.gradient) * 2)

True